In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mlrose
import time

from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split  
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

scaler = MinMaxScaler()

dataset_train = pd.read_csv("~/Documents/Datasets/ML/fifa-processed-train.csv")
dataset_test = pd.read_csv("~/Documents/Datasets/ML/fifa-processed-test.csv")

X_train = dataset_train[['Weak Foot', 'Skill Moves', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing','Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']]
y_train = dataset_train['position_label']
X_test = dataset_test[['Weak Foot', 'Skill Moves', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing','Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']]
y_test = dataset_test['position_label']

In [ ]:
# Scaler to transform input data since NNs are sensitive to attributes
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
one_hot = OneHotEncoder(categories='auto')
le = LabelEncoder()
y_train_hot = one_hot.fit_transform(y_train.values.reshape(-1, 1)).todense()
y_test_hot = one_hot.transform(y_test.values.reshape(-1, 1)).todense()

In [ ]:
def plot_acc_iters(iterations, train_accuracy, test_accuracy):
    plt.figure()
    plt.title("NN Weight Optimization")
    plt.xlabel("Maximum Iterations")
    plt.ylabel("Accuracy")
    plt.grid()
    
    plt.plot(iterations, train_accuracy, 'o-', color="r",label="Train")
    plt.plot(iterations, test_accuracy, 'o-', color="g",label="Test")
    plt.legend(loc="best")
    return plt

def plot_acc_time(iterations, train_accuracy, test_accuracy):
    plt.figure()
    plt.title("NN Weight Optimization")
    plt.xlabel("Time")
    plt.ylabel("Accuracy")
    plt.grid()
    
    plt.plot(iterations, train_accuracy, 'o-', color="r",label="Train")
    # plt.plot(iterations, test_accuracy, 'o-', color="g",label="Test")
    plt.legend(loc="best")
    return plt

In [ ]:
def plot_cumTime(time_train_rhc, time_test_rhc, time_train_simA, time_test_simA, time_train_ga, time_test_ga, train_accuracy_rhc, test_accuracy_rhc, train_accuracy_simA, test_accuracy_simA, train_accuracy_ga, test_accuracy_ga):
    plt.figure()
    plt.title("Algorithm Comparison")
    plt.xlabel("Time")
    plt.ylabel("Accuracy")
    plt.grid()
    
    plt.plot(time_train_rhc, train_accuracy_rhc, 'o-', color="r",label="RHC Train")
    plt.plot(time_train_simA, train_accuracy_simA, 'o-', color="g",label="SimA Train")
    plt.plot(time_train_ga, train_accuracy_ga, 'o-', color="b",label="GA Train")
    # plt.plot(iterations, test_accuracy, 'o-', color="g",label="Test")
    plt.legend(loc="best")
    return plt

In [ ]:
# RHC for ANN FIFA (OLD, Does not support time-graphs, still useful for accuracy graphs)

np.random.seed(3)

no_of_iterations = [10,20,50,100,200,500,1000,2000,4000,5000,6000,8000,10000]
time_for_iteration_train = []
time_for_iteration_test = []
train_accuracy = []
test_accuracy = []
train_error = []
test_error = []

for i in no_of_iterations:
    start_time_train = time.time()
    print("iteration: ", i)
    nn_model = mlrose.NeuralNetwork(hidden_nodes = [50], activation = 'relu', 
                                    algorithm = 'random_hill_climb', max_iters = i, 
                                    is_classifier = True, learning_rate = 0.5, bias = True,
                                    early_stopping = False, max_attempts = 50)
    
    nn_model.fit(X_train, y_train_hot)
    time_for_iteration_train.append(time.time() - start_time_train)
    
    print("Predicting training data")
    y_train_pred = nn_model.predict(X_train)
    y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
    y_train_error = mean_squared_error(y_train_hot, y_train_pred)
    
    print("Predicting testing data")
    start_time_test = time.time()
    y_test_pred = nn_model.predict(X_test)
    time_for_iteration_test.append(time.time() - start_time_test)
    y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
    y_test_error = mean_squared_error(y_test_hot, y_test_pred)
    
    train_error.append(y_train_error)
    test_error.append(y_test_error)
    train_accuracy.append(y_train_accuracy)
    test_accuracy.append(y_test_accuracy)
    
print(time_for_iteration_train)

In [ ]:
plot_acc_iters(no_of_iterations, train_accuracy, test_accuracy)

In [ ]:
# SimA for FIFA (OLD, Does not support time-graphs, still useful for accuracy graphs)

np.random.seed(3)

no_of_iterations = [10,20,50,100,200,500,1000,2000,4000,5000,6000,8000,10000]
time_for_iteration_train = []
time_for_iteration_test = []
train_accuracy = []
test_accuracy = []
train_error = []
test_error = []

for i in no_of_iterations:
    start_time_train = time.time()
    print("iteration: ", i)
    nn_model = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu', 
                                    algorithm = 'simulated_annealing', max_iters = i, 
                                    is_classifier = True, learning_rate = 0.5, 
                                    early_stopping = False, max_attempts = 100)
    
    nn_model.fit(X_train, y_train_hot)
    time_for_iteration_train.append(time.time() - start_time_train)
    
    print("Predicting training data")
    y_train_pred = nn_model.predict(X_train)
    y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
    y_train_error = mean_squared_error(y_train_hot, y_train_pred)
    
    print("Predicting testing data")
    start_time_test = time.time()
    y_test_pred = nn_model.predict(X_test)
    time_for_iteration_test.append(time.time() - start_time_test)
    y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
    y_test_error = mean_squared_error(y_test_hot, y_test_pred)
    
    train_error.append(y_train_error)
    test_error.append(y_test_error)
    train_accuracy.append(y_train_accuracy)
    test_accuracy.append(y_test_accuracy)
    
print(time_for_iteration_train)

In [ ]:
plot_acc_iters(no_of_iterations, train_accuracy, test_accuracy)

In [ ]:
plot_acc_iters(no_of_iterations, train_accuracy, test_accuracy)

In [ ]:
# GenAlg for FIFA (OLD, Does not support time-graphs, still useful for accuracy graphs)

np.random.seed(3)

no_of_iterations = [10,20,50,100,200,500,1000,2000,5000]
time_for_iteration_train = []
time_for_iteration_test = []
train_accuracy = []
test_accuracy = []
train_error = []
test_error = []

for i in no_of_iterations:
    start_time_train = time.time()
    print("iteration: ", i)
    nn_model = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu', 
                                    algorithm = 'genetic_alg', max_iters = i, pop_size = 100,
                                    is_classifier = True, learning_rate = 0.5, clip_max = 1,
                                    early_stopping = True, max_attempts = 100)
    
    nn_model.fit(X_train, y_train_hot)
    time_for_iteration_train.append(time.time() - start_time_train)
    
    print("Predicting training data")
    y_train_pred = nn_model.predict(X_train)
    y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
    y_train_error = mean_squared_error(y_train_hot, y_train_pred)
    
    print("Predicting testing data")
    start_time_test = time.time()
    y_test_pred = nn_model.predict(X_test)
    time_for_iteration_test.append(time.time() - start_time_test)
    y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
    y_test_error = mean_squared_error(y_test_hot, y_test_pred)
    
    train_error.append(y_train_error)
    test_error.append(y_test_error)
    train_accuracy.append(y_train_accuracy)
    test_accuracy.append(y_test_accuracy)
    
print(time_for_iteration_train)

In [ ]:
plot_acc_iters(no_of_iterations, train_accuracy, test_accuracy)

In [ ]:
# GenAlg for FIFA (OLD, Does not support time-graphs, still useful for accuracy graphs)

np.random.seed(3)

no_of_iterations = [10,20,50,100,150,200,250,300,350,400,450,500,600,700,800,900,1000]
time_for_iteration_train = []
time_for_iteration_test = []
train_accuracy = []
test_accuracy = []
train_error = []
test_error = []

for i in no_of_iterations:
    start_time_train = time.time()
    print("iteration: ", i)
    nn_model = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu', 
                                    algorithm = 'genetic_alg', max_iters = i, pop_size = 200, mutation_prob = 0.25,
                                    is_classifier = True, learning_rate = 0.5, clip_max = 1,
                                    early_stopping = True, max_attempts = 10)
    
    nn_model.fit(X_train, y_train_hot)
    time_for_iteration_train.append(time.time() - start_time_train)
    
    print("Predicting training data")
    y_train_pred = nn_model.predict(X_train)
    y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
    y_train_error = mean_squared_error(y_train_hot, y_train_pred)
    
    print("Predicting testing data")
    start_time_test = time.time()
    y_test_pred = nn_model.predict(X_test)
    time_for_iteration_test.append(time.time() - start_time_test)
    y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
    y_test_error = mean_squared_error(y_test_hot, y_test_pred)
    
    train_error.append(y_train_error)
    test_error.append(y_test_error)
    train_accuracy.append(y_train_accuracy)
    test_accuracy.append(y_test_accuracy)
    
print(time_for_iteration_train)

In [ ]:
plot_acc_iters(no_of_iterations, train_accuracy, test_accuracy)

In [ ]:
## FROM here on, supports time graphs!

## RHC NN, many iterations
np.random.seed(3)

no_of_iterations = [10,20,50,100,200,500,1000,2000,4000,5000,6000,8000,10000]
time_train_rhc = []
time_test_rhc = []
train_accuracy_rhc = []
test_accuracy_rhc = []

for i in no_of_iterations:
    start_time_train = time.time()
    print("iteration: ", i)
    nn_model = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu', 
                                    algorithm = 'random_hill_climb', max_iters = i, 
                                    is_classifier = True, learning_rate = 0.2, bias = True,
                                    early_stopping = False, max_attempts = 50)
    
    nn_model.fit(X_train, y_train_hot)
    time_train_rhc.append(time.time() - start_time_train)
    
    print("Predicting training data")
    y_train_pred = nn_model.predict(X_train)
    y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
    y_train_error = mean_squared_error(y_train_hot, y_train_pred)
    
    print("Predicting testing data")
    start_time_test = time.time()
    y_test_pred = nn_model.predict(X_test)
    time_test_rhc.append(time.time() - start_time_test)
    y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
    y_test_error = mean_squared_error(y_test_hot, y_test_pred)
    
    train_accuracy_rhc.append(y_train_accuracy)
    test_accuracy_rhc.append(y_test_accuracy)

In [ ]:
# SimA NN, many iterations
time_train_simA = []
time_test_simA = []
train_accuracy_simA = []
test_accuracy_simA = []

for i in no_of_iterations:
    start_time_train = time.time()
    print("iteration: ", i)
    nn_model = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu', 
                                    algorithm = 'simulated_annealing', max_iters = i, 
                                    is_classifier = True, learning_rate = 0.5,
                                    early_stopping = False, max_attempts = 50)
    
    nn_model.fit(X_train, y_train_hot)
    time_train_simA.append(time.time() - start_time_train)
    
    print("Predicting training data")
    y_train_pred = nn_model.predict(X_train)
    y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
    y_train_error = mean_squared_error(y_train_hot, y_train_pred)
    
    print("Predicting testing data")
    start_time_test = time.time()
    y_test_pred = nn_model.predict(X_test)
    time_test_simA.append(time.time() - start_time_test)
    y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
    y_test_error = mean_squared_error(y_test_hot, y_test_pred)
    
    train_accuracy_simA.append(y_train_accuracy)
    test_accuracy_simA.append(y_test_accuracy)

In [ ]:
# GA NN, many iterations
time_train_ga = []
time_test_ga = []
train_accuracy_ga = []
test_accuracy_ga = []

for i in no_of_iterations:
    start_time_train = time.time()
    print("iteration: ", i)
    nn_model = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu', 
                                    algorithm = 'genetic_alg', max_iters = i, pop_size = 200, mutation_prob = 0.25,
                                    is_classifier = True, learning_rate = 0.5, clip_max = 1,
                                    early_stopping = True, max_attempts = 10)
    
    nn_model.fit(X_train, y_train_hot)
    time_train_ga.append(time.time() - start_time_train)
    
    print("Predicting training data")
    y_train_pred = nn_model.predict(X_train)
    y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
    y_train_error = mean_squared_error(y_train_hot, y_train_pred)
    
    print("Predicting testing data")
    start_time_test = time.time()
    y_test_pred = nn_model.predict(X_test)
    time_test_ga.append(time.time() - start_time_test)
    y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
    y_test_error = mean_squared_error(y_test_hot, y_test_pred)
    
    train_accuracy_ga.append(y_train_accuracy)
    test_accuracy_ga.append(y_test_accuracy)

In [ ]:
plot_cumTime(time_train_rhc, time_test_rhc, time_train_simA, time_test_simA, time_train_ga, time_test_ga, train_accuracy_rhc, test_accuracy_rhc, train_accuracy_simA, test_accuracy_simA, train_accuracy_ga, test_accuracy_ga)

In [ ]:
# ---------------------------------------------------------------------------------------

In [ ]:
## THIS IS REALLY OLD, TEST CODE

np.random.seed(3)

nn_model1 = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu',
                                 algorithm = 'random_hill_climb', max_iters = 10000,
                                 is_classifier = True, learning_rate = 0.01,
                                 early_stopping = True, clip_max = 5, max_attempts = 100)

# nn_model1.fit(X_train, y_train_hot)

In [ ]:
def plot_final_curve_rhc():
    plt.figure()
    plt.title("NN Graph - RHC")
    plt.xlabel("Percentage of training set")
    plt.ylabel("Error")
    plt.grid()
    
    x_axis_values = []
    y_train_error_graph = []
    y_test_error_graph = []
    y_train_accuracy_graph = []
    y_test_accuracy_graph = []
    
    for i in range(1,11):
        print ("in iteration: ", i)
        X_train_selected = X_train[0:int((i/10)*len(X_train))]
        y_train_selected = y_train_hot[0:int((i/10)*len(y_train))]
        nn_model1.fit(X_train_selected, y_train_selected)

        y_train_pred = nn_model1.predict(X_train)
        y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
        y_train_error = mean_squared_error(y_train_hot, y_train_pred)

        y_test_pred = nn_model1.predict(X_test)
        y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
        y_test_error = mean_squared_error(y_test_hot, y_test_pred)

        x_axis_values.append(i/10)
        y_train_error_graph.append(y_train_error)
        y_test_error_graph.append(y_test_error)
        y_train_accuracy_graph.append(y_train_accuracy)
        y_test_accuracy_graph.append(y_test_accuracy)
    
    plt.plot(x_axis_values, y_train_error_graph, 'o-', color="r",label="Training error")
    plt.plot(x_axis_values, y_test_error_graph, 'o-', color="g",label="Testing error")
    plt.legend(loc="best")
    plt.figure()
    plt.title("NN Graph - RHC")
    plt.xlabel("Percentage of training set")
    plt.ylabel("Accuracy")
    plt.grid()
    plt.plot(x_axis_values, y_train_accuracy_graph, 'o-', color="b",label="Training accuracy")
    plt.plot(x_axis_values, y_test_accuracy_graph, 'o-', color="y",label="Testing accuracy")

start_time = time.time()
print(start_time)
plot_final_curve_rhc()
end_time = time.time()
print("time for execution is:", end_time - start_time)
#print(X_train[0])

In [ ]:
np.random.seed(3)

nn_model2 = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu',
                                 algorithm = 'genetic_alg', max_iters = 10000, pop_size = 500, mutation_prob = 0.1,
                                 bias = False, is_classifier = True, learning_rate = 0.001,
                                 early_stopping = True, max_attempts = 500)

In [ ]:
def plot_final_curve_GA():
    plt.figure()
    plt.title("NN Graph - GA")
    plt.xlabel("Percentage of training set")
    plt.ylabel("Error")
    plt.grid()
    
    x_axis_values = []
    y_train_error_graph = []
    y_test_error_graph = []
    y_train_accuracy_graph = []
    y_test_accuracy_graph = []
    
    for i in range(1,11):
        print("in iteration: ", i)
        X_train_selected = X_train[0:int((i/10)*len(X_train))]
        y_train_selected = y_train_hot[0:int((i/10)*len(y_train))]
        nn_model2.fit(X_train_selected, y_train_selected)

        y_train_pred = nn_model2.predict(X_train)
        y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
        y_train_error = mean_squared_error(y_train_hot, y_train_pred)

        y_test_pred = nn_model2.predict(X_test)
        y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
        y_test_error = mean_squared_error(y_test_hot, y_test_pred)

        x_axis_values.append(i/10)
        y_train_error_graph.append(y_train_error)
        y_test_error_graph.append(y_test_error)
        y_train_accuracy_graph.append(y_train_accuracy)
        y_test_accuracy_graph.append(y_test_accuracy)
    
    plt.plot(x_axis_values, y_train_error_graph, 'o-', color="r",label="Training error")
    plt.plot(x_axis_values, y_test_error_graph, 'o-', color="g",label="Testing error")
    plt.legend(loc="best")
    
    # plt.plot(x_axis_values, y_train_accuracy_graph, 'o-', color="b",label="Training accuracy")
    # plt.plot(x_axis_values, y_test_accuracy_graph, 'o-', color="y",label="Testing accuracy")
start_time = time.time()
print(start_time)
plot_final_curve_GA()
end_time = time.time()
print("time for execution is:", end_time - start_time)


In [ ]:
np.random.seed(3)

nn_model2 = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu',
                                 algorithm = 'genetic_alg', max_iters = 10000, pop_size = 200, mutation_prob = 0.1,
                                 bias = False, is_classifier = True, learning_rate = 0.001,
                                 early_stopping = True, clip_max = 1, max_attempts = 100)

In [ ]:
start_time = time.time()
print(start_time)
def plot_final_curve_GA():
    plt.figure()
    plt.title("NN Graph - GA")
    plt.xlabel("Percentage of training set")
    plt.ylabel("Error")
    plt.grid()
    
    x_axis_values = []
    y_train_error_graph = []
    y_test_error_graph = []
    y_train_accuracy_graph = []
    y_test_accuracy_graph = []
    
    for i in range(1,11):
        print("in iteration: ", i)
        X_train_selected = X_train[0:int((i/10)*len(X_train))]
        y_train_selected = y_train_hot[0:int((i/10)*len(y_train))]
        nn_model2.fit(X_train_selected, y_train_selected)

        y_train_pred = nn_model2.predict(X_train)
        y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
        y_train_error = mean_squared_error(y_train_hot, y_train_pred)

        y_test_pred = nn_model2.predict(X_test)
        y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
        y_test_error = mean_squared_error(y_test_hot, y_test_pred)

        x_axis_values.append(i/10)
        y_train_error_graph.append(y_train_error)
        y_test_error_graph.append(y_test_error)
        y_train_accuracy_graph.append(y_train_accuracy)
        y_test_accuracy_graph.append(y_test_accuracy)
        print("time for iteration is:", time.time() - start_time)
    
    plt.plot(x_axis_values, y_train_error_graph, 'o-', color="r",label="Training error")
    plt.plot(x_axis_values, y_test_error_graph, 'o-', color="g",label="Testing error")
    plt.legend(loc="best")
    
    # plt.plot(x_axis_values, y_train_accuracy_graph, 'o-', color="b",label="Training accuracy")
    # plt.plot(x_axis_values, y_test_accuracy_graph, 'o-', color="y",label="Testing accuracy")

plot_final_curve_GA()
end_time = time.time()
print("time for execution is:", end_time - start_time)


In [ ]:
np.random.seed(3)

nn_model3 = mlrose.NeuralNetwork(hidden_nodes = [10,10,10], activation = 'relu',
                                 algorithm = 'simulated_annealing', max_iters = 10000, schedule = mlrose.ExpDecay(),
                                 bias = False, is_classifier = True, learning_rate = 0.001,
                                 early_stopping = True, max_attempts = 100)

In [ ]:
start_time = time.time()
print(start_time)
def plot_final_curve_SimA():
    plt.figure()
    plt.title("NN Graph - SimA")
    plt.xlabel("Percentage of training set")
    plt.ylabel("Error")
    plt.grid()
    
    x_axis_values = []
    y_train_error_graph = []
    y_test_error_graph = []
    y_train_accuracy_graph = []
    y_test_accuracy_graph = []
    
    for i in range(1,11):
        print("in iteration: ", i)
        X_train_selected = X_train[0:int((i/10)*len(X_train))]
        y_train_selected = y_train_hot[0:int((i/10)*len(y_train))]
        nn_model3.fit(X_train_selected, y_train_selected)

        y_train_pred = nn_model3.predict(X_train)
        y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)
        y_train_error = mean_squared_error(y_train_hot, y_train_pred)

        y_test_pred = nn_model3.predict(X_test)
        y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)
        y_test_error = mean_squared_error(y_test_hot, y_test_pred)

        x_axis_values.append(i/10)
        y_train_error_graph.append(y_train_error)
        y_test_error_graph.append(y_test_error)
        y_train_accuracy_graph.append(y_train_accuracy)
        y_test_accuracy_graph.append(y_test_accuracy)
        print("time for iteration is:", time.time() - start_time)
    
    plt.plot(x_axis_values, y_train_error_graph, 'o-', color="r",label="Training error")
    plt.plot(x_axis_values, y_test_error_graph, 'o-', color="g",label="Testing error")
    plt.legend(loc="best")
    
    # plt.plot(x_axis_values, y_train_accuracy_graph, 'o-', color="b",label="Training accuracy")
    # plt.plot(x_axis_values, y_test_accuracy_graph, 'o-', color="y",label="Testing accuracy")

plot_final_curve_SimA()
end_time = time.time()
print("time for execution is:", end_time - start_time)